In [7]:
# !pip install transformers datasets torch torchvision opencv-python kaggle

# Download Dataset

In [8]:
# import os

# input_path = "/kaggle/input"
# for root, dirs, files in os.walk(input_path):
#     for file in files:
#         if file.endswith(".zip"):
#             print(os.path.join(root, file))


In [9]:
# !pip install kagglehub

In [10]:

# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("hasyimabdillah/workoutfitness-video")

# print("Path to dataset files:", path)

In [11]:
import os
import glob
import torch
import cv2
from torch.utils.data import Dataset
from torchvision import transforms

class VideoDataset(Dataset):
    def __init__(self, root_dir, transform=None, frames_per_video=8):
        self.root_dir = root_dir
        self.transform = transform
        self.frames_per_video = frames_per_video
        self.video_paths = glob.glob(os.path.join(root_dir, '*/*.mp4'))
        self.label_map = {label: idx for idx, label in enumerate(sorted(os.listdir(root_dir)))}

    def __len__(self):
        return len(self.video_paths)

    def read_video_frames(self, video_path):
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_idxs = torch.linspace(0, total_frames - 1, self.frames_per_video).long().tolist()
        frames = []

        for i in range(total_frames):
            ret, frame = cap.read()
            if not ret:
                break
            if i in frame_idxs:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                if self.transform:
                    frame = self.transform(frame)
                frames.append(frame)

        cap.release()
        return torch.stack(frames)  # shape: (T, C, H, W)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = os.path.basename(os.path.dirname(video_path))
        video_tensor = self.read_video_frames(video_path)
        return video_tensor, self.label_map[label]


# Define Transformer and DataLoader

In [12]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = VideoDataset(r"F:\quest_digiflex\exp\data\archive", transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)


# Load TimeSfomer from Hugging Face

In [13]:
from transformers import AutoModelForVideoClassification, AutoImageProcessor

model = AutoModelForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")
processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")

# Adjust number of classes if needed
model.classifier = torch.nn.Linear(model.classifier.in_features, len(dataset.label_map))


In [ ]:
# Fine-tune the Model

In [14]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")


In [15]:
dataset_val = VideoDataset(r"F:\quest_digiflex\exp\test\test", transform=transform)
dataloader_val = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)

In [16]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=0)


In [17]:
from torch.utils.data import DataLoader

val_loader = DataLoader(dataset_val, batch_size=2, shuffle=True, num_workers=0)


In [18]:
import torch
from transformers import AutoModelForVideoClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")
model.classifier = torch.nn.Linear(model.classifier.in_features, len(dataset.label_map))  # Adjust num classes
model.to(device)


TimesformerForVideoClassification(
  (timesformer): TimesformerModel(
    (embeddings): TimesformerEmbeddings(
      (patch_embeddings): TimesformerPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (time_drop): Dropout(p=0.0, inplace=False)
    )
    (encoder): TimesformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x TimesformerLayer(
          (drop_path): Identity()
          (attention): TimeSformerAttention(
            (attention): TimesformerSelfAttention(
              (qkv): Linear(in_features=768, out_features=2304, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
            )
            (output): TimesformerSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): TimesformerIntermediate(
            (dense

In [19]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)


In [20]:
from transformers import TimesformerConfig, TimesformerModel
import torch.nn as nn

class TimeSformerWithDropout(nn.Module):
    def __init__(self, num_classes):
        super(TimeSformerWithDropout, self).__init__()
        self.backbone = TimesformerModel.from_pretrained("facebook/timesformer-base-finetuned-k400")
        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_classes)

    def forward(self, video):
        outputs = self.backbone(video)
        x = self.dropout(outputs.last_hidden_state[:, 0])  # CLS token
        logits = self.classifier(x)
        return logits


In [21]:
from torchvision import transforms
import random

frame_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.2, contrast=0.2)], p=0.5),
    transforms.RandomAffine(degrees=10, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [22]:
import os

dataset_root = r"F:\quest_digiflex\exp\data\archive"  # your dataset folder
num_classes = len(next(os.walk(dataset_root))[1])
print("Detected classes:", num_classes)


Detected classes: 22


In [23]:
# !pip install tqdm


In [24]:
# import torch
# import torch.nn as nn
# import torch.optim as optim

# num_classes = 22  # <-- Add this line

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = TimeSformerWithDropout(num_classes=num_classes).to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.AdamW(model.parameters(), lr=1e-4)
# num_epochs = 10


# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     correct = 0
#     total = 0

#     for i, (videos, labels) in enumerate(train_loader):
#         videos, labels = videos.to(device), labels.to(device)

#         logits = model(videos)
#         loss = criterion(logits, labels)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         preds = logits.argmax(dim=1)
#         correct += (preds == labels).sum().item()
#         total += labels.size(0)

#     acc = correct / total
#     print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {running_loss:.4f} | Accuracy: {acc:.4f}")


In [ ]:
# num_epochs = 5

# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     correct = 0
#     total = 0

#     for i, (videos, labels) in enumerate(train_loader):
#         # videos: (B, T, C, H, W) → TimeSformer expects (B, T, C, H, W)
#         videos, labels = videos.to(device), labels.to(device)

#         outputs = model(videos)
#         loss = criterion(outputs.logits, labels)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Track metrics
#         running_loss += loss.item()
#         preds = outputs.logits.argmax(dim=1)
#         correct += (preds == labels).sum().item()
#         total += labels.size(0)

#     acc = correct / total
#     print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {running_loss:.4f} | Accuracy: {acc:.4f}")


Epoch [1/5] | Loss: 401.3763 | Accuracy: 0.6661
Epoch [2/5] | Loss: 47.1124 | Accuracy: 0.9831
Epoch [3/5] | Loss: 12.4966 | Accuracy: 0.9966
Epoch [4/5] | Loss: 5.0691 | Accuracy: 1.0000
Epoch [5/5] | Loss: 3.2462 | Accuracy: 1.0000


In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from tqdm import tqdm
# import copy

# # Constants
# num_classes = 22
# num_epochs = 20
# patience = 3  # for early stopping
# best_val_loss = float('inf')
# early_stop_counter = 0

# # Device setup
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Initialize model
# model = TimeSformerWithDropout(num_classes=num_classes).to(device)

# # Loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# # To track best model
# best_model_wts = copy.deepcopy(model.state_dict())

# # Training loop
# for epoch in range(num_epochs):
#     print(f"\n🔁 Epoch {epoch+1}/{num_epochs}")

#     # === Train ===
#     model.train()
#     running_loss = 0.0
#     correct = 0
#     total = 0

#     train_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Training")
#     for i, (videos, labels) in train_bar:
#         videos, labels = videos.to(device), labels.to(device)

#         optimizer.zero_grad()
#         logits = model(videos)
#         loss = criterion(logits, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         preds = logits.argmax(dim=1)
#         correct += (preds == labels).sum().item()
#         total += labels.size(0)

#         if i % 5 == 0:
#             print(f"📦 Train Batch {i}/{len(train_loader)} | Loss: {loss.item():.4f}")

#     train_acc = correct / total
#     train_loss = running_loss / len(train_loader)

#     # === Validation ===
#     model.eval()
#     val_loss = 0.0
#     val_correct = 0
#     val_total = 0

#     with torch.no_grad():
#         val_bar = tqdm(enumerate(val_loader), total=len(val_loader), desc="Validating")
#         for i, (videos, labels) in val_bar:
#             videos, labels = videos.to(device), labels.to(device)
#             logits = model(videos)
#             loss = criterion(logits, labels)

#             val_loss += loss.item()
#             preds = logits.argmax(dim=1)
#             val_correct += (preds == labels).sum().item()
#             val_total += labels.size(0)

#     val_acc = val_correct / val_total
#     val_loss /= len(val_loader)

#     # === Logging ===
#     print(f"✅ Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

#     # === Early Stopping ===
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         best_model_wts = copy.deepcopy(model.state_dict())
#         early_stop_counter = 0
#         print("🔖 Best model updated.")
#     else:
#         early_stop_counter += 1
#         print(f"⚠️ No improvement in validation loss for {early_stop_counter} epoch(s).")

#     if early_stop_counter >= patience:
#         print("⛔ Early stopping triggered.")
#         break

# # Load best weights
# model.load_state_dict(best_model_wts)

# # Save best model
# # torch.save(model.state_dict(), "best_timesformer_model.pth")
# print("✅ Best model saved to 'best_timesformer_model.pth'")



🔁 Epoch 1/20


Training:   0%|▏                                                                     | 1/295 [00:49<4:04:49, 49.97s/it]

📦 Train Batch 0/295 | Loss: 3.4782


Training:   2%|█▍                                                                    | 6/295 [03:50<2:58:41, 37.10s/it]

📦 Train Batch 5/295 | Loss: 0.9098


...

📦 Train Batch 45/295 | Loss: 0.1114


Training:  17%|███████████▉                                                         | 51/295 [28:38<2:31:01, 37.14s/it]

📦 Train Batch 50/295 | Loss: 0.0616


Training:  19%|█████████████                                                        | 56/295 [31:24<2:21:41, 35.57s/it]

📦 Train Batch 55/295 | Loss: 0.1719


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import copy
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import os

# Constants
num_classes = 22
num_epochs = 20
patience = 3
best_val_loss = float('inf')
early_stop_counter = 0

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model
model = TimeSformerWithDropout(num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# TensorBoard setup
writer = SummaryWriter(log_dir="runs/timesformer_experiment")

# Best model tracking
best_model_wts = copy.deepcopy(model.state_dict())

# Class names (optional, if available)
# class_names = [str(i) for i in range(num_classes)]

for epoch in range(num_epochs):
    print(f"\n🔁 Epoch {epoch+1}/{num_epochs}")

    # === Train ===
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    train_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training")
    for i, (videos, labels) in train_bar:
        videos, labels = videos.to(device), labels.to(device)

        optimizer.zero_grad()
        logits = model(videos)
        loss = criterion(logits, labels)
        loss.backward()

        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        running_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        if i % 5 == 0:
            print(f"📦 Train Batch {i}/{len(train_loader)} | Loss: {loss.item():.4f}")

    train_acc = correct / total
    train_loss = running_loss / len(train_loader)

    # TensorBoard: log training metrics
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)

    # === Validation ===
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        val_bar = tqdm(enumerate(val_loader), total=len(val_loader), desc="Validating")
        for i, (videos, labels) in val_bar:
            videos, labels = videos.to(device), labels.to(device)
            logits = model(videos)
            loss = criterion(logits, labels)

            val_loss += loss.item()
            preds = logits.argmax(dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_acc = val_correct / val_total
    val_loss /= len(val_loader)

    # TensorBoard: log validation metrics
    writer.add_scalar('Loss/val', val_loss, epoch)
    writer.add_scalar('Accuracy/val', val_acc, epoch)

    # === Logging ===
    print(f"✅ Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # === Confusion Matrix & Per-Class Accuracy ===
    if epoch == num_epochs - 1 or val_loss < best_val_loss:
        cm = confusion_matrix(all_labels, all_preds)
        print("\n📊 Confusion Matrix:")
        print(cm)

        report = classification_report(all_labels, all_preds, digits=4)
        print("\n📄 Classification Report:")
        print(report)

        # TensorBoard: log confusion matrix as image (optional)
        try:
            import matplotlib.pyplot as plt
            import seaborn as sns

            fig = plt.figure(figsize=(10, 8))
            sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
            plt.xlabel("Predicted")
            plt.ylabel("True")
            plt.title("Confusion Matrix")
            writer.add_figure("Confusion Matrix", fig, epoch)
            plt.close(fig)
        except ImportError:
            pass  # If matplotlib/seaborn not installed, skip plotting

    # === Early Stopping ===
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        early_stop_counter = 0
        print("🔖 Best model updated.")
    else:
        early_stop_counter += 1
        print(f"⚠️ No improvement in validation loss for {early_stop_counter} epoch(s).")

    if early_stop_counter >= patience:
        print("⛔ Early stopping triggered.")
        break

# Load best weights
model.load_state_dict(best_model_wts)

# Save best model
# torch.save(model.state_dict(), "best_timesformer_model.pth")
print("✅ Best model saved to 'best_timesformer_model.pth'")

# Close TensorBoard writer
writer.close()



🔁 Epoch 1/20


Training:   0%|▏                                                                     | 1/295 [00:37<3:03:32, 37.46s/it]

📦 Train Batch 0/295 | Loss: 4.2632


Training:   2%|█▍                                                                    | 6/295 [03:35<2:47:03, 34.69s/it]

📦 Train Batch 5/295 | Loss: 2.5579


...

📦 Train Batch 210/295 | Loss: 2.2708


Training:  73%|████████████████████████████████████████████████▎                 | 216/295 [2:33:08<1:18:38, 59.73s/it]

📦 Train Batch 215/295 | Loss: 5.2342


Training:  73%|█████████████████████████████████████████████████▊                  | 216/295 [2:36:06<57:05, 43.36s/it]


RuntimeError: stack expects each tensor to be equal size, but got [7, 3, 224, 224] at entry 0 and [8, 3, 224, 224] at entry 1

In [ ]:
 # r"F:\quest_digiflex\exp\data\archive"

In [1]:
import os
import cv2
import torch
import random
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# -------------------------------
# Configuration
# -------------------------------
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

VIDEO_DIR =  r"F:\quest_digiflex\exp\data\archive"
NUM_FRAMES = 16
IMAGE_SIZE = 112
BATCH_SIZE = 4
NUM_EPOCHS = 30
PATIENCE = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------
# Utils
# -------------------------------
def read_video(path, num_frames):
    cap = cv2.VideoCapture(path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < num_frames:
        return None  # skip too-short videos

    indices = np.linspace(0, total_frames - 1, num_frames).astype(int)
    i = 0
    count = 0
    while cap.isOpened() and count < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if i in indices:
            frame = cv2.resize(frame, (IMAGE_SIZE, IMAGE_SIZE))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
            count += 1
        i += 1
    cap.release()
    return np.array(frames)

# -------------------------------
# Dataset
# -------------------------------
class VideoDataset(Dataset):
    def __init__(self, file_paths, labels, num_frames=16, augment=True):
        self.file_paths = file_paths
        self.labels = labels
        self.num_frames = num_frames
        self.augment = augment
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.RandomHorizontalFlip(p=0.5) if augment else transforms.Lambda(lambda x: x),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        video_path = self.file_paths[idx]
        label = self.labels[idx]
        frames = read_video(video_path, self.num_frames)

        if frames is None:
            return self.__getitem__((idx + 1) % len(self.file_paths))  # try next

        frames = [self.transform(frame) for frame in frames]
        video_tensor = torch.stack(frames).permute(1, 0, 2, 3)  # (C, T, H, W)
        return video_tensor, label

# -------------------------------
# Model
# -------------------------------
class Simple3DCNN(nn.Module):
    def __init__(self, num_classes):
        super(Simple3DCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2),

            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2),

            nn.Conv3d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool3d((1, 1, 1))
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.classifier(self.features(x))

# -------------------------------
# Data Preparation
# -------------------------------
def load_data():
    class_to_idx = {cls: idx for idx, cls in enumerate(os.listdir(VIDEO_DIR))}
    file_paths = []
    labels = []
    for cls in class_to_idx:
        folder = os.path.join(VIDEO_DIR, cls)
        for file in os.listdir(folder):
            if file.endswith(".mp4"):
                file_paths.append(os.path.join(folder, file))
                labels.append(class_to_idx[cls])
    return file_paths, labels, len(class_to_idx)

file_paths, labels, num_classes = load_data()
train_files, val_files, train_labels, val_labels = train_test_split(
    file_paths, labels, test_size=0.2, random_state=SEED, stratify=labels
)

train_dataset = VideoDataset(train_files, train_labels, NUM_FRAMES, augment=True)
val_dataset = VideoDataset(val_files, val_labels, NUM_FRAMES, augment=False)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# -------------------------------
# Training
# -------------------------------
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience):
    best_acc = 0.0
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        val_correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in tqdm(val_loader, desc="Validating"):
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == targets).sum().item()
                total += targets.size(0)

        val_acc = val_correct / total
        print(f"Epoch {epoch+1}: Train Loss={running_loss/len(train_loader):.4f}, Val Acc={val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_model.pt")
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print("Early stopping triggered.")
                break

    print("Training complete. Best Val Acc:", best_acc)

# -------------------------------
# Run Everything
# -------------------------------
model = Simple3DCNN(num_classes).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

train_model(model, train_loader, val_loader, criterion, optimizer, NUM_EPOCHS, PATIENCE)


Validating: 100%|██████████████████████████████████████████████████████████████████████| 30/30 [02:56<00:00,  5.87s/it]


Epoch 1: Train Loss=2.9983, Val Acc=0.1695


Validating: 100%|██████████████████████████████████████████████████████████████████████| 30/30 [02:45<00:00,  5.51s/it]


Epoch 2: Train Loss=2.9370, Val Acc=0.1949


Validating: 100%|██████████████████████████████████████████████████████████████████████| 30/30 [02:42<00:00,  5.43s/it]


...

Epoch 10: Train Loss=2.7538, Val Acc=0.2119


Validating: 100%|██████████████████████████████████████████████████████████████████████| 30/30 [02:46<00:00,  5.54s/it]


Epoch 11: Train Loss=2.7396, Val Acc=0.2119


Validating: 100%|██████████████████████████████████████████████████████████████████████| 30/30 [02:53<00:00,  5.77s/it]

Epoch 12: Train Loss=2.6601, Val Acc=0.1949
Early stopping triggered.
Training complete. Best Val Acc: 0.211864406779661


In [ ]:
import os
import torch

# Custom save path (Windows-style path with escaped backslashes or raw string)
custom_path = r"F:\quest_digiflex\exp\devanshi\model\best_model.pth"

# Ensure directory exists
os.makedirs(os.path.dirname(custom_path), exist_ok=True)

# # Inside your training loop, after validation:
# if val_loss < best_val_loss:
#     best_val_loss = val_loss
torch.save(model.state_dict(), custom_path)
print(f"🔖 Best model saved at: {custom_path}")

In [ ]:
# Save the full model (recommended for ease)
torch.save(model, "timesformer_exercise_model.pth")


In [ ]:
import pickle

# Save the full model object (must be in the same session when loading)
with open("timesformer_model.pkl", "wb") as f:
    pickle.dump(model, f)


In [ ]:
import pickle

with open("timesformer_model.pkl", "rb") as f:
    model = pickle.load(f)

model.eval()
model.to(device)


TimesformerForVideoClassification(
  (timesformer): TimesformerModel(
    (embeddings): TimesformerEmbeddings(
      (patch_embeddings): TimesformerPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (time_drop): Dropout(p=0.0, inplace=False)
    )
    (encoder): TimesformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x TimesformerLayer(
          (drop_path): Identity()
          (attention): TimeSformerAttention(
            (attention): TimesformerSelfAttention(
              (qkv): Linear(in_features=768, out_features=2304, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
            )
            (output): TimesformerSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): TimesformerIntermediate(
            (dense

In [ ]:
def predict(video_tensor):
    model.eval()
    with torch.no_grad():
        inputs = video_tensor.unsqueeze(0).to(device)
        outputs = model(inputs)
        predicted_class = outputs.logits.argmax(dim=1).item()
        return predicted_class


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model (if not already loaded)
# model = torch.load("timesformer_exercise_model.pth")
# model.to(device)

# Reverse label map
idx_to_label = {v: k for k, v in dataset.label_map.items()}

video_path = "sample_video.mp4"
predicted_class = predict_from_video_path(video_path, model, device, idx_to_label)
print(f"Predicted class: {predicted_class}")
